In [ ]:
import pandas as pd
import re
import persian
import pickle
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans


df = pd.read_excel(r'...\pishnahdkamel.XLS' )
df = df[pd.notnull(df['sharhpishnahad'])]  #حذف سطرهایی که ستون پیشنهادات آن خالی است
temp = df[df.sharhpishnahad.duplicated()==False]#حذف پیشنهادهای تکراری

#کل اطلاعات ستون پیشنهادات را درکنارهم نمایش میدهد
print(temp.loc[0, 'sharhpishnahad'])

temp["original"] = temp.sharhpishnahad

stopwords = []
file = open('stopwords.txt', encoding = 'utf-8').read()
[stopwords.append(x) for x in file.split()]
stopwords = set(stopwords)

#برای حذف لغات زاید بکار میرود درواقع جایگذاری باخالی میشود
def remove_general_stopwords(text):
    text = str(text)
    filtered_tokens = [token for token in text.split() if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#برای تبدیل حروف ی و ک بکار میرود
def convert_ar_chars(text):
    s = ''
    for word in text:
        s = s + persian.convert_ar_characters(word)
    return s

temp.sharhpishnahad = temp.sharhpishnahad.apply(remove_general_stopwords)

temp.sharhpishnahad= temp.sharhpishnahad.apply(convert_ar_chars)
temp.sharhpishnahad = temp.sharhpishnahad.apply(remove_general_stopwords)

#تعداد تکرار هرکلمه در ستون پیشنهاد را نشان میدهد
w_count = temp.sharhpishnahad.str.split(expand=True).stack().value_counts()


X = temp.sharhpishnahad

#کلماتی که حداقل تعدادتکرارآنها 100 است را باروش tfidf تبدیل به یک آرایه کن
# هریک از لغات درواقع یک هنصرآرایه است وبه مدل نگاشته میشود
model = Pipeline([('vect', CountVectorizer(min_df=100)),
                  ('tfidf', TfidfTransformer(sublinear_tf=True))])
XX = model.fit_transform(X)

#چون هریک از لغات به عنوان یک ویژگی است
model.named_steps['vect'].get_feature_names()

#از دیتای ساخته شده برای مدل یک دیتافریم ساخته میشود
new_df = pd.DataFrame(XX.toarray(),
                        columns = model.named_steps['vect'].get_feature_names())



kmeans = KMeans(n_clusters = 3, random_state=1)
kmeans.fit(new_df)

# temp['labels'] = kmeans.labels_
temp.loc[:,'labels'] = kmeans.labels_

temp.labels.value_counts()


sse = {}
for k in range(2, 10):
    kmeans_s = KMeans(n_clusters=k, max_iter=100).fit(new_df)
    sse[k] = kmeans_s.inertia_
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()


#برای ذخیره مدل استفاده میشود که برای دفعات بعدی مدل لود شود ویک متن به عنوان ورودی داده شود وبراساس مدل فیت شده قبلی بتواند جواب را استخراج کند
with open('kmeans.pickle','wb') as f:
    pickle.dump(kmeans, f)
with open('kmeans.pickle','rb') as f:
    clf = pickle.load(f)

sample = """
نصب دستگاه کالیبراسیون درخط تولیدبرای اندازه گیری آنلاین
"""

sample = model.transform([sample]).toarray()
sentiment = clf.predict(sample)